In [227]:
import pandas as pd
import numpy as np
from __future__ import division
import calendar
import time
import datetime
from datetime import date, timedelta
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
from dateutil.parser import parse
from pandas.tslib import Timestamp
colorz=sns.color_palette("husl", 8)
#sns.palplot(sns.color_palette("husl", 8))
import matplotlib.dates as mdates

In [44]:
path='/Users/stella/Desktop/'
date_0=datetime.date(2017,4,14)
date_1=datetime.date(2017,7,1)

In [48]:
# PEGASOR INDOOR AND OUTDOOR DATA
i_ldsa=pd.read_excel(path+'data-2/realeco_sn0135-ldsa_2017-01-23t181000_2017-07-01t181000.xlsx')
i_ldsa=i_ldsa.set_index('time')
i_pn=pd.read_excel(path+'data-2/realeco_sn0135-pm25_2017-01-19t181600_2017-07-01t181600.xlsx')
i_pn=i_pn.set_index('time')
i_pm25=pd.read_excel(path+'data-2/realeco_sn0135-pn_2017-01-25t183900_2017-07-01t183900.xlsx')
i_pm25=i_pm25.set_index('time')

i_df=(i_ldsa.join(i_pn)).join(i_pm25).dropna()
i_df.columns=['i-ldsa','i-pm25','i-pn']

urban=pd.read_csv(path+'data-2/1329_PEGASOR A_20170712185105-2.csv',delimiter=',')
urban['DateTime']=urban['DateTime'].apply(lambda x: parse(x))
urban.columns=['time','ldsa-1','pn-1','pm25-1','ldsa-2','pn-2','pm25-2']
urban['pn-1'].loc[:80034]=urban['pn-1'].loc[:80034]*6
urban['pn-2'].loc[:80033]=urban['pn-2'].loc[:80033]*6
urban=urban.set_index('time')

pegasor=i_df.join(urban).reset_index()

pegasor=pegasor[(pegasor['time']>date_0) & (pegasor['time']<date_1)]
pegasor=pegasor[pegasor>0].dropna().reset_index()
pegasor=pegasor.reset_index()
del pegasor['index']
del pegasor['level_0']

pegasor['ldsa-1/2']=pegasor['ldsa-1']/pegasor['ldsa-2']
pegasor['ldsa-2/1']=pegasor['ldsa-2']/pegasor['ldsa-1']

pegasor=pegasor[pegasor['ldsa-2/1']<5]
pegasor=pegasor[pegasor['ldsa-1/2']<5].reset_index()
del pegasor['index'], pegasor['ldsa-1/2'], pegasor['ldsa-2/1']

times=pd.DatetimeIndex(pegasor.time)
peg_grouped=pegasor.groupby([times.month, times.day, times.hour]).mean().reset_index()
peg_grouped=peg_grouped.rename(columns={'level_0':'month','level_1':'day','level_2':'hour'})
peg_grouped['year']=2017
peg_grouped['date']=pd.to_datetime(peg_grouped[['year','month','day','hour']])
del peg_grouped['year'], peg_grouped['month'], peg_grouped['day'], peg_grouped['hour']

/Users/stella/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [49]:
#april         3783
apr_3783=pd.read_excel(path+'queens-data/Queens Data Logger TSI 3783 Ultrafine February to April.xls')[1:-8]
apr_3783['Date&Time']=apr_3783['Date&Time'].apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y %I:%M %p'))

#may-june      3783
may_jun_3783=pd.read_csv(path+'queens-data/TEOM_TSI 3783_May-June_2017.csv')[:-8]
may_jun_3783 = may_jun_3783.rename(columns=may_jun_3783.iloc[1])
may_jun_3783 = may_jun_3783[3:]
may_jun_3783=may_jun_3783[['Date&Time','UF Aggregated Conc.']]
may_jun_3783['Date&Time']=may_jun_3783['Date&Time'].apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y %I:%M %p'))

tsi_3783=pd.concat([apr_3783,may_jun_3783])

tsi_3783['UF Aggregated Conc.']=pd.to_numeric(tsi_3783['UF Aggregated Conc.'], errors='coerce')
tsi_3783=tsi_3783[(tsi_3783['Date&Time']>date_0) & (tsi_3783['Date&Time']<date_1)]
tsi_3783=tsi_3783.reset_index()
del tsi_3783['index']

tsi_3783.columns=['time','TNC (#/cm3)']

tsi_3783=tsi_3783[tsi_3783['TNC (#/cm3)']>5]

times=pd.DatetimeIndex(tsi_3783.time)
tsi_3783_grouped=tsi_3783.groupby([times.month, times.day, times.hour]).mean().reset_index()
tsi_3783_grouped=tsi_3783_grouped.rename(columns={'level_0':'month','level_1':'day','level_2':'hour'})
tsi_3783_grouped['year']=2017
tsi_3783_grouped['date']=pd.to_datetime(tsi_3783_grouped[['year','month','day','hour']])
del tsi_3783_grouped['year'], tsi_3783_grouped['month'], tsi_3783_grouped['day'], tsi_3783_grouped['hour']

In [50]:
#april         3031
apr_3031=pd.read_csv(path+'queens-data/TSI 3031 Particle Counter_20170413-april.TXT',delimiter='\t',skiprows=4)
apr_3031=apr_3031.reset_index()
apr_3031.columns=apr_3031.iloc[0]
apr_3031=apr_3031.iloc[1:].reset_index()
apr_3031['start']=apr_3031['start'].apply(lambda x: parse(x))
apr_3031['end']=apr_3031['end'].apply(lambda x: parse(x))
apr_3031=apr_3031[['start','end','ch1','ch2','ch3','ch4','ch5','ch6']]

#may-june      3031
may_jun_3031=pd.read_csv(path+'queens-data/TSI 3031_20170501.TXT',delimiter='\t',skiprows=4)
may_jun_3031=may_jun_3031.reset_index()
may_jun_3031.columns=may_jun_3031.iloc[0]
may_jun_3031=may_jun_3031.iloc[1:].reset_index()
may_jun_3031['start']=may_jun_3031['start'].apply(lambda x: parse(x))
may_jun_3031['end']=may_jun_3031['end'].apply(lambda x: parse(x))
may_jun_3031=may_jun_3031[['start','end','ch1','ch2','ch3','ch4','ch5','ch6']]

tsi_3031=pd.concat([apr_3031,may_jun_3031])

tsi_3031=tsi_3031[(tsi_3031['start']>date_0) & (tsi_3031['start']<date_1)]
tsi_3031[['ch1','ch2','ch3','ch4','ch5','ch6']] = tsi_3031[['ch1','ch2','ch3','ch4','ch5','ch6']].apply(pd.to_numeric)
tsi_3031=tsi_3031[tsi_3031.ch1>0]
tsi_3031=tsi_3031.reset_index()
del tsi_3031['index']

times=pd.DatetimeIndex(tsi_3031.start)
tsi_3031_grouped=tsi_3031.groupby([times.month, times.day, times.hour]).mean().reset_index()
tsi_3031_grouped=tsi_3031_grouped.rename(columns={'level_0':'month','level_1':'day','level_2':'hour'})
tsi_3031_grouped['year']=2017
tsi_3031_grouped['date']=pd.to_datetime(tsi_3031_grouped[['year','month','day','hour']])
del tsi_3031_grouped['year'], tsi_3031_grouped['month'], tsi_3031_grouped['day'], tsi_3031_grouped['hour']

In [51]:
tsi_3783_grouped=tsi_3783_grouped.set_index('date')
tsi_3031_grouped=tsi_3031_grouped.set_index('date')
peg_grouped=peg_grouped.set_index('date')

df=tsi_3783_grouped.join(tsi_3031_grouped).join(peg_grouped).dropna().reset_index()
df['smps-sum']=df['ch1']+df['ch2']+df['ch3']+df['ch4']+df['ch5']+df['ch6']

df_apr=df[df.date<datetime.date(2017,5,1)]
df_may=df[(df.date>=datetime.date(2017,5,1)) & (df.date<datetime.date(2017,6,1))]
df_jun=df[(df.date>=datetime.date(2017,6,1)) & (df.date<datetime.date(2017,7,1))]

# JUL - SEP

In [52]:
path2='/Users/stella/Desktop/CHTechUSA Data_2017_1002/'

In [53]:
df1=pd.read_csv(path2+'Data Logger Download.csv')
df1=df1[['Date&Time','PM2.5_FEM_MC','PM10_MC','UF Aggregated Conc.']][1:132480].reset_index()
del df1['index']
df1['Date&Time']=df1['Date&Time'].apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y %I:%M %p'))
df1.columns=['time','PM2.5 (ug/m3)','PM10 (ug/m3)','TNC (#/cm3)']
df1['PM2.5 (ug/m3)']=pd.to_numeric(df1['PM2.5 (ug/m3)'], errors='coerce')
df1['PM10 (ug/m3)']=pd.to_numeric(df1['PM10 (ug/m3)'], errors='coerce')
df1['TNC (#/cm3)']=pd.to_numeric(df1['TNC (#/cm3)'], errors='coerce')

times=pd.DatetimeIndex(df1.time)
df1_grouped=df1.groupby([times.month, times.day]).mean().reset_index()
df1_grouped=df1_grouped.rename(columns={'level_0':'month','level_1':'day'})
df1_grouped['year']=2017
df1_grouped['date']=pd.to_datetime(df1_grouped[['year','month','day']])
del df1_grouped['year'], df1_grouped['month'], df1_grouped['day']

In [54]:
df2=pd.read_csv(path2+'TEOM-1405DF_A225741308_2017-10-2_13-0-27.csv')
df2 = df2.rename(columns=df2.iloc[2])[3:].reset_index()
del df2['index']
df2 = df2[['Time Stamp','PM-2.5 FEM MC','PM-10 1-Hr MC']]
df2.columns=['time','PM2.5-teom (ug/m3)','PM10-teom (ug/m3)']
df2['time']=df2['time'].apply(lambda x: parse(x))
df2['PM2.5-teom (ug/m3)']=pd.to_numeric(df2['PM2.5-teom (ug/m3)'], errors='coerce')
df2['PM10-teom (ug/m3)']=pd.to_numeric(df2['PM10-teom (ug/m3)'], errors='coerce')

times=pd.DatetimeIndex(df2.time)
df2_grouped=df2.groupby([times.month, times.day, times.hour]).mean().reset_index()
df2_grouped=df2_grouped.rename(columns={'level_0':'month','level_1':'day'})
df2_grouped['year']=2017
df2_grouped['date']=pd.to_datetime(df2_grouped[['year','month','day']])
del df2_grouped['year'], df2_grouped['month'], df2_grouped['day']

In [55]:
df3=pd.read_csv(path2+'TSI3031_20170701.TXT',delimiter='\t')
df3=df3[['start','end','ch1','ch2','ch3','ch4','ch5','ch6']]
df3['start']=df3['start'].apply(lambda x: parse(x))
df3['end']=df3['end'].apply(lambda x: parse(x))

times=pd.DatetimeIndex(df3.start)
df3_grouped=df3.groupby([times.month, times.day, times.hour]).mean().reset_index()
df3_grouped=df3_grouped.rename(columns={'level_0':'month','level_1':'day'})
df3_grouped['year']=2017 
df3_grouped['date']=pd.to_datetime(df3_grouped[['year','month','day']])
del df3_grouped['year'], df3_grouped['month'], df3_grouped['day']

In [56]:
df=df1_grouped.set_index('date').join(df3_grouped.set_index('date')).reset_index()

In [57]:
# PEGASOR INDOOR SN-135 FROM REALECO
ildsa=pd.read_excel(path2+'realeco_sn0135-ldsa_2017-06-30t144100_2017-07-14t144100.xlsx')
ipm=pd.read_excel(path2+'realeco_sn0135-pm25_2017-06-30t145200_2017-07-14t145200.xlsx')
ipn=pd.read_excel(path2+'realeco_sn0135-pn_2017-06-30t145300_2017-07-14t145300.xlsx')
idf=ildsa.set_index('time').join(ipm.set_index('time')).join(ipn.set_index('time')).dropna().reset_index()
idf['time']=idf['time']-datetime.timedelta(hours=1)          # TIME ADJUSTED -1 HOUR

idf=idf[(idf['time']>=datetime.date(2017,7,1)) & (idf['time']<=datetime.date(2017,7,14))].reset_index()
del idf['index']

In [58]:
df.columns

Index(['date', 'PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'TNC (#/cm3)', 'level_2',
       'ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6'],
      dtype='object')

In [59]:
df=df[df['PM2.5 (ug/m3)']>0].reset_index()
del df['index']

In [60]:
df=df1_grouped.set_index('date').join(df3_grouped.set_index('date')).reset_index()

In [64]:
times=pd.DatetimeIndex(df1.time)
df1_hr=df1.groupby([times.month, times.day, times.hour]).mean().reset_index()
df1_hr=df1_hr.rename(columns={'level_0':'month','level_1':'day','level_2':'hour'})
df1_hr['year']=2017
df1_hr['date']=pd.to_datetime(df1_hr[['year','month','day','hour']])
del df1_hr['year'], df1_hr['month'], df1_hr['day'], df1_hr['hour']

In [65]:
df1_hr=df1_hr[df1_hr.date<datetime.date(2017,8,1)]        #teom, 3783 JULY

In [66]:
times=pd.DatetimeIndex(df3.start)
df3_hr=df3.groupby([times.month, times.day, times.hour]).mean().reset_index()
df3_hr=df3_hr.rename(columns={'level_0':'month','level_1':'day','level_2':'hour'})
df3_hr['year']=2017
df3_hr['date']=pd.to_datetime(df3_hr[['year','month','day','hour']])
del df3_hr['year'], df3_hr['month'], df3_hr['day'], df3_hr['hour']

In [67]:
df3_hr=df3_hr[df3_hr.date<datetime.date(2017,8,1)]       #smps JULY

In [320]:
urban=pd.read_csv('/Users/stella/Desktop/CHTechUSA Data_2017_1002/Pagesor A_29 June to 02 Oct 2017_UTC0.csv',delimiter=',',encoding='latin-1')
urban=urban[['Alarm','Data','pegasor a','pegasor a.1','pegasor a.2','pegasor a.19','pegasor a.20','pegasor a.21','pegasor a.37','pegasor a.38','pegasor a.39']]
urban.columns=['alarm','data','ldsa-1','pn-1','pm25-1','ldsa-2','pn-2','pm25-2','ldsa-3','pn-3','pm25-3']

/Users/stella/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [321]:
urban=urban[7:].reset_index()

In [322]:
urban.head()

,index,alarm,data,ldsa-1,pn-1,pm25-1,ldsa-2,pn-2,pm25-2,ldsa-3,pn-3,pm25-3
0,7,29/06/2017,00:01,30.47,176784.55,1.21,0.00,0.00,0.00,34.04,194934.49,1.34
1,8,29/06/2017,00:02,30.47,176784.55,1.21,0.00,0.00,0.00,34.04,194934.49,1.34
2,9,29/06/2017,00:03,30.47,176784.55,1.21,0.00,0.00,0.00,33.93,194266.39,1.33
3,10,29/06/2017,00:04,30.47,176784.55,1.21,0.00,0.00,0.00,33.93,194266.39,1.33
4,11,29/06/2017,00:05,30.47,176784.55,1.21,0.00,0.00,0.00,35.71,204585.48,1.40


In [324]:
urban['alarm']=urban['alarm'].apply(lambda x: datetime.datetime.strptime(x,'%d/%m/%Y'))

In [325]:
urban['alarm'][urban['data']=='24:00']=urban['alarm'][urban['data']=='24:00']+timedelta(1)
urban['data'][urban['data']=='24:00']='00:00'

/Users/stella/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/stella/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [ ]:
urban['datetime']=urban['alarm']+' '+urban['data']
urban['datetime'].apply(lambda x: datetime.datetime.strptime(x,'%d/%m/%Y %H:%M'))
#del urban['alarm'],urban['data']

In [331]:
time.strptime(urban['data'][0],'%H:%M')

time.struct_time(tm_year=1900, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=1, tm_sec=0, tm_wday=0, tm_yday=1, tm_isdst=-1)

In [ ]:
urban['alarm']

In [336]:
urban.head()

,index,alarm,data,ldsa-1,pn-1,pm25-1,ldsa-2,pn-2,pm25-2,ldsa-3,pn-3,pm25-3
0,7,2017-06-29,00:01,30.47,176784.55,1.21,0.00,0.00,0.00,34.04,194934.49,1.34
1,8,2017-06-29,00:02,30.47,176784.55,1.21,0.00,0.00,0.00,34.04,194934.49,1.34
2,9,2017-06-29,00:03,30.47,176784.55,1.21,0.00,0.00,0.00,33.93,194266.39,1.33
3,10,2017-06-29,00:04,30.47,176784.55,1.21,0.00,0.00,0.00,33.93,194266.39,1.33
4,11,2017-06-29,00:05,30.47,176784.55,1.21,0.00,0.00,0.00,35.71,204585.48,1.40


In [332]:
urban['data'].apply(lambda x: time.strptime(x,'%H:%M'))

0           (1900, 1, 1, 0, 1, 0, 0, 1, -1)
1           (1900, 1, 1, 0, 2, 0, 0, 1, -1)
2           (1900, 1, 1, 0, 3, 0, 0, 1, -1)
3           (1900, 1, 1, 0, 4, 0, 0, 1, -1)
4           (1900, 1, 1, 0, 5, 0, 0, 1, -1)
5           (1900, 1, 1, 0, 6, 0, 0, 1, -1)
6           (1900, 1, 1, 0, 7, 0, 0, 1, -1)
7           (1900, 1, 1, 0, 8, 0, 0, 1, -1)
8           (1900, 1, 1, 0, 9, 0, 0, 1, -1)
9          (1900, 1, 1, 0, 10, 0, 0, 1, -1)
10         (1900, 1, 1, 0, 11, 0, 0, 1, -1)
11         (1900, 1, 1, 0, 12, 0, 0, 1, -1)
12         (1900, 1, 1, 0, 13, 0, 0, 1, -1)
13         (1900, 1, 1, 0, 14, 0, 0, 1, -1)
14         (1900, 1, 1, 0, 15, 0, 0, 1, -1)
15         (1900, 1, 1, 0, 16, 0, 0, 1, -1)
16         (1900, 1, 1, 0, 17, 0, 0, 1, -1)
17         (1900, 1, 1, 0, 18, 0, 0, 1, -1)
18         (1900, 1, 1, 0, 19, 0, 0, 1, -1)
19         (1900, 1, 1, 0, 20, 0, 0, 1, -1)
20         (1900, 1, 1, 0, 21, 0, 0, 1, -1)
21         (1900, 1, 1, 0, 22, 0, 0, 1, -1)
22         (1900, 1, 1, 0, 23, 0